In [1]:
#识别手写系统MNIST

输入图片，然后模型处理，之后识别结果

# 过程
定义超参数,构建transforms，对图像做变换,下载数据集,
构建网络模型,
定义训练方法,
定义测试方法,
开始训练模型，输出结果

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
#定义超参数
BATCH_SIZE = 32 #分批次，减少电脑负担
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")#决定是gpu训练是cpu训练
EPOCHS = 10  #训练的轮次

In [4]:
#构建pipeline，对图像进行处理
pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))    #过拟合时降低模型复杂度
])

In [5]:
#下载加载数据
from torch.utils.data import DataLoader
#下载
train_set = datasets.MNIST("data_mnist", train=True, download=True, transform=pipeline) 

test_set = datasets.MNIST("data_mnist", train=False, download=True, transform=pipeline)
#
train_loader = DataLoader(train_set, batch_size= BATCH_SIZE, shuffle=True) #shuffle是打乱的意思

test_loader = DataLoader(test_set, batch_size= BATCH_SIZE, shuffle=True)


C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
#查看，显示图片
with open(r"C:\Users\user\DL\Torch_study\data_mnist\MNIST\raw\train-images-idx3-ubyte","rb") as f:
    file=f.read()

In [7]:
image1 = [int(str(item).encode('ascii'),16) for item in file[16 : 16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [8]:
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28,28,1)

print(image1_np.shape)

(28, 28, 1)


In [9]:
cv2.imwrite("MNIST.jpg",image1_np)

True

In [10]:
#构建网络
class Digital(nn.Module):
    def __init__(self):
        super().__init__()
        #卷积层一般作为滤波器 
        self.conv1 = nn.Conv2d(1,10,5) # 1代表灰度图片的通道，10表示输出通道。5是kernal卷积核
        self.conv2 = nn.Conv2d(10,20,3) #10是输入通道，20输出，3是kernal
        #官方代码有Dropout，丢了一部分，nn.Dropout(0.25)
        self.fc1 = nn.Linear(20*10*10, 500) #全连接层，第一个是输入通道，第二个是输出通道
        self.fc2 = nn.Linear(500, 10) #输出10个概率？
    def forward(self,x):
        input_size = x.size(0) #batch_size x1 x 28x 28
        x = self.conv1(x) #输入 x1 x 28x 28， 输出是batch*10*24*24
        #10是输出通道，24是28-5+1=24
        x = F.relu(x) #激活函数 在所有隐藏层支架加入激活函数，让输出是非线性，让表达能力更强，
        #softmax函数多分类，sigmoid二分类
        x = F.max_pool2d(x,2,2) #池化层，将图片压缩，降采样
        #最大池化层 重点看最大的区域
        #这里输入是 10*24*24 输出是 10*12*12
        
        x = self.conv2(x) #10*12*12 输出变成 20*10*10   10是因为12-kernal（3）+1
        x = F.relu(x)
        
        #官方用的flatten，拉平，torch.flatten
        x = x.view(input_size,-1) #拉伸， -1表示自动计算维度 其实应该是20*10*10=2000
        
        #送入全连接层
        x = self.fc1(x) #输入2000 输出500
        x = F.relu(x) #保持shape不变
        
        x = self.fc2(x) #最后得到10的输出
        
        output = F.log_softmax(x,dim=1) #计算分类后，每个数字的概率值，损失函数
        
        return output

In [11]:
#定义优化器
model = Digital().to(DEVICE)

optimizer = optim.Adam(model.parameters())


In [12]:
#定义训练方法
def train_model(model, device, train_loader,optimizer, epoch):
    #模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader): #enumerate可以给出下标
        #部署到devices
        data, target = data.to(device),target.to(device)
        #梯度初始化
        optimizer.zero_grad()
        #预测，训练后的结果
        output = model(data)
        #计算损失，即比较预测和真实结果
        loss = F.cross_entropy(output,target) #交叉熵针对多分类任务，
        #找到概率值最大的下标,后面可以用来计算准确率
        pred = output.max(1,keepdim=True)#pred = output.argmax(dim=1)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch: {} \t Loss: {:.6f}".format(epoch,loss.item()))
        

In [13]:
#定义测试方法
def test_model(model,device,test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss =0.0
    with torch.no_grad(): #不会计算梯度，也不会进行反向传播
        for data,target in test_loader:
            #部署到device上
            data,target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            #找到概率最大的下标
            pred = output.max(1,keepdim=True)[1] #0是值，1是索引
            #pred = torch.max(output,dim=1)
            #pred = output.argmax(dim=1)
            #累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test_average_Loss: {:.4f},Accuracy:{:.3f}\n".format(
            test_loss, 100.0*correct/len(test_loader.dataset)))

In [14]:
#调用方法
for epoch in range(1,EPOCHS+1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

C:\Users\user\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 	 Loss: 2.294851
Test_average_Loss: 0.0018,Accuracy:98.100

Train Epoch: 2 	 Loss: 0.037167
Test_average_Loss: 0.0012,Accuracy:98.920

Train Epoch: 3 	 Loss: 0.147214
Test_average_Loss: 0.0013,Accuracy:98.690

Train Epoch: 4 	 Loss: 0.002908
Test_average_Loss: 0.0015,Accuracy:98.550

Train Epoch: 5 	 Loss: 0.002023
Test_average_Loss: 0.0014,Accuracy:98.710

Train Epoch: 6 	 Loss: 0.001296
Test_average_Loss: 0.0013,Accuracy:98.980

Train Epoch: 7 	 Loss: 0.000015
Test_average_Loss: 0.0014,Accuracy:98.960

Train Epoch: 8 	 Loss: 0.000789
Test_average_Loss: 0.0016,Accuracy:98.910

Train Epoch: 9 	 Loss: 0.000088
Test_average_Loss: 0.0017,Accuracy:98.790

Train Epoch: 10 	 Loss: 0.000186
Test_average_Loss: 0.0013,Accuracy:99.060

